# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-t1bfb2fq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-t1bfb2fq
  Resolved https://github.com/huggingface/accelerate.git to commit 828aae4e32f1d06e84d9436cd2204a861dbcd1c4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.2.0.dev0-py3-none-any.whl size=336774 sha256=5929f572a5f5f37675a6082e32417a5d7fc5d43c56f9e40e5ead71000fd2f42d
  Stored in directory: /tmp/pip-ephem-wheel-cache-u0o23vcm/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
 

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [ ]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [ ]:
sp_nl2sql = sp_nl2sql.format(question="Return The name of the best paid employee")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee 

In [ ]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [ ]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Response
    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    `{question}`:
    ```sql3
    """


In [ ]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [ ]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Samples

    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: `Return the names of the three people who have had the highest salary increase in the last three years.`
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
    GROUP BY e.name
    ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
    LIMIT 3;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    GROUP BY e.name
    ORDER BY MAX(s.salary) DESC
    LIMIT 1;


#Now the question in spanish.


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Devuelveme el nombre del empleado mejor pagado")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.year >= DATE_SUB(CURDATE(), INTERVAL 3 YEAR)
     GROUP BY e.name
     ORDER BY (MAX(s.salary) - MIN(s.salary)) DESC
     LIMIT 3;


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

# **Test same prompt (sp_nl2sql3b) with a different question in spanish**

In [ ]:
sp_nl2sql4 = sp_nl2sql3b.format(question="Devuelveme el salario del empleado con menor nivel educativo")
print (sp_nl2sql4)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql4, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, s.educational_level
     FROM employees e
     JOIN studies s ON e.ID_Usr = s.ID_Usr
     WHERE s.educational_level > 3
     ORDER BY e.name NULLS LAST, s.educational_level NULLS LAST;


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

# **Test same prompt (sp_nl2sql3b) with the same previous question but translated to english**

In [ ]:
sp_nl2sql5 = sp_nl2sql3b.format(question="Return the salary of the employee with the lowest educational level.")
print (sp_nl2sql5)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql5, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT MIN(s.salary) AS lowest_salary
    FROM employees e
    JOIN studies s ON e.ID_Usr = s.ID_Usr
    WHERE s.educational_level = MIN(s.educational_level);


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# **New context prompt from sp_nl2sql2 optimization**

In [9]:
sp_nl2sql6 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,  -- 5=phd, 4=Master, 3=Bachelor
        Institution VARCHAR, --Name of instituon where eployee studied
        Years DATE, -- Date acomplishement stdy
        Speciality VARCHAR, -- Speciality of studies
        primary key (ID_study, ID_Usr), --Primary Key ID_Usr + ID_Study
        foreign key(ID_Usr) references employees (ID_Usr)
        );

    ### Samples
    Question: "How many employees have a salary greater than 50,000?"
    SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    WHERE s.salary > 50000;

    Question: "Return the names and salaries of employees with a salary greater than 75,000 and an educational level of Master or higher."
    SELECT e.name, s.salary
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    JOIN studies t ON e.ID_Usr = t.ID_Usr
    WHERE s.salary > 75000 AND t.educational_level >= 4;

    Question: "Return the average salary of employees who studied in institutions with 'University' in their name."
    SELECT AVG(s.salary) AS avg_salary
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    JOIN studies t ON e.ID_Usr = t.ID_Usr
    WHERE t.Institution LIKE '%University%';

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question: `{question}`:
    `{question}`:
    ```sql3
    """


# **Tryout 1: Multi-condition Query in Spanish**

In [ ]:
sp_nl2sql6a = sp_nl2sql6.format(question="Devuélveme el nombre y el salario de los empleados que tienen un nivel educativo mayor a 3 y que ganan más de 50,000.")
print (sp_nl2sql6a)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql6a, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT COUNT(*) AS total_employees
    FROM employees e
    INNER JOIN salary s ON e.ID_Usr = s.ID_Usr
    INNER JOIN studies t ON e.ID_Usr = t.ID_Usr
    WHERE t.educational_level > 3 AND s.salary > 50000;


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

# **Tryout 2: Aggregate Functions**



In [ ]:
sp_nl2sql6b = sp_nl2sql6.format(question="Return the average salary of employees who studied at institutions with the word 'University' in their name.")
print (sp_nl2sql6b)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [ ]:
input_sentences = tokenizer(sp_nl2sql6b, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT AVG(s.salary) AS avg_salary
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_Usr
    JOIN studies t ON e.ID_Usr = t.ID_Usr
    WHERE t.institution LIKE '%University%';


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

# **Tryout 3: Temporal Conditions**

In [10]:
sp_nl2sql6c = sp_nl2sql6.format(question="Return the names of employees who were hired in the last 5 years.")
print (sp_nl2sql6c)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   create table employees(
        ID_Usr INT primary key,-- Unique Id for employee
        name VARCHAR -- Name of employee
        );

    create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    create table studies(
        ID_study INT, -- Unique ID study
        ID_Usr INT, -- ID employee
        educational_level INT,

In [11]:
input_sentences = tokenizer(sp_nl2sql6c, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [12]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, to_char(s.year, 'YYYY-MM-DD') AS hire_date
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     JOIN studies t ON e.ID_Usr = t.ID_Usr
     WHERE to_date(to_char(s.year, 'YYYY-MM-DD'), 'YYYY-MM-DD') > CURRENT_DATE - interval '5 years';


In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

# **Lab Report: NL2SQL**

Report: Natural Language to SQL Using AI Models
________________________________________
A.	Objective

The objective of this lab was to evaluate and understand how a language model performs in generating SQL queries from natural language prompts. Specific goals included testing the model’s ability to handle multilingual prompts, complex SQL logic, and varied query types. The lab also aimed to explore the model’s strengths, weaknesses, and areas for improvement, particularly in its ability to handle examples and contextual instructions.

B.	Findings
1. Sensitivity to Language

•	The model demonstrated better accuracy when prompts were provided in English compared to Spanish.

•	In Spanish, the model often misunderstood the intent, leading to incorrect or irrelevant SQL queries. For example:

o	The query for "Devuélveme el nombre del empleado mejor pagado" generated results unrelated to finding the highest-paid employee.

o	In contrast, the English version produced a valid query that correctly joined tables and used MAX(salary) to identify the highest-paid employee.

2. Variations That Didn't Work Well

•	The model struggled with:

o	Multilingual prompts: Spanish queries often resulted in hallucinations or irrelevant logic.

o	Advanced SQL logic: In English, it produced partial solutions, such as comparing MIN() values directly in a WHERE clause, which is invalid SQL syntax.

o	Over-reliance on examples (shots): Including examples in prompts sometimes biased the model, causing it to follow patterns irrelevant to the specific query.

3. Comparison of Initial Context Prompts

•	Three different initial contexts were compared:

1.	Without examples (sp_nl2sql): Focused purely on schema and instructions. Results were inconsistent for complex queries.

2.	With examples (sp_nl2sql2): Included two SQL query examples (shots), improving accuracy but introducing biases towards those patterns.

3.	Enhanced examples (sp_nl2sql3b): Similar to the second but with more explicit instructions to use the examples. Slightly better performance but no significant difference from sp_nl2sql2.

•	Conclusion: The second context (sp_nl2sql2) was chosen as the base for optimization due to its balance between examples and clarity. However, diversity in examples is needed to avoid overfitting to specific logic patterns.

4. Results from Tryouts

Tryout 1: Multi-condition Query

•	Prompt: "Devuélveme el nombre y el salario de los empleados que tienen un nivel educativo mayor a 3 y que ganan más de 50,000."

•	Evaluation: Partially correct. The query misinterpreted the prompt and generated a COUNT(*) instead of selecting names and salaries. However, conditions (AND) were correctly implemented.

Tryout 2: Aggregate Functions

•	Prompt: "Return the average salary of employees who studied at institutions with the word 'University' in their name."

•	Evaluation: Correct. The query used the AVG() function and LIKE clause appropriately, producing the desired result without errors.

Tryout 3: Temporal Conditions

•	Prompt: "Return the names of employees who were hired in the last 5 years."

•	Evaluation: Partially correct. The query included unnecessary conversions (to_char and to_date), making it overly complex. The temporal logic (CURRENT_DATE - interval '5 years') was correct.

C.	What I Learned

•	Handling Conditions: The model can interpret combined conditions (AND) correctly but may fail to return the correct columns if the prompt lacks specificity.

•	Aggregate Functions: It performs well with functions like AVG() and textual filters (LIKE), especially in English.

•	Multilingual Challenges: While the model handles basic conditions in Spanish, it shows greater precision in English.

•	Example Dependency: Examples included in prompts significantly influence the model’s outputs.

D.	Future Considerations

•	Improving Multilinguality: Train the model with more examples in Spanish to equal its performance in English.

•	Query Validation: Implement a post-processing layer to validate the syntax and logic of generated queries.

•	Diverse Examples: Include examples covering advanced SQL functions, such as windowing, subqueries, and clean temporal logic.

E.	Most Value-Added Concepts

•	Prompt Optimization: Learning how to craft precise and clear prompts significantly impacts the quality of the output.

•	Model Behavior Analysis: Understanding how the model processes multilingual input and examples provides insights for refining its usage.

•	SQL Post-Processing: Identifying the necessity of validating SQL outputs ensures reliability for real-world applications.

F.	Conclusions

1.	English Dominance:

o	The model performs best with prompts in English, suggesting it is better fine-tuned for this language.

o	Multilingual capabilities need further improvement for broader usability.

2.	Dependence on Examples:

o	While examples help guide the model, they can overly influence the output, leading to unintended patterns in SQL logic.

o	Examples should be carefully curated to ensure diverse coverage without introducing biases.

3.	Validation Required:

o	Post-generation validation is essential to ensure SQL queries are logically and syntactically correct.

o	Errors like comparing MIN() in a WHERE clause indicate the need for additional checks.

4.	Lab Objective Achievement:

o	The lab successfully evaluated the model's ability to translate natural language into SQL queries across different languages and query complexities. While the model showed strengths in English and with aggregate functions, it revealed limitations in multi-language and handling complex temporal logic.

By addressing these areas, the model’s usability and reliability for real-world applications can be significantly improved.
